In [19]:
import os
import datetime as dt
import json 
import math

if "src" not in os.listdir():
    os.chdir("../../../")


%load_ext autoreload
%autoreload 2

from src.lib.alpaca_paper import AlpacaTrader
from src.lib.alpaca_historical import AlpacaData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
def update_holdings(alp_trader, desired_holdings):
    # Get the total account equity
    equity = float(alp_trader.get_account().json()["equity"])
    
    # Get current account positions
    positions = alp_trader.get_positions().json()

    # Find the price of every relevant symbol
    prices = {}
    # All symbols currently being held
    for position in positions:
        prices[position["symbol"]] = float(position["current_price"])

    # All desired symbols excluding already held symbols
    for symbol in desired_holdings:
        if symbol not in prices:
            prices[symbol] = float(alp_trader.get_current_price(symbol).json()["latestTrade"]["p"])
    
    # Find the current holdings as a percentage of account equity
    current_holdings = {}
    current_shares = {}
    for position in positions:
        symbol = position["symbol"]
        qty = float(position["qty"])
        current_shares[symbol] = qty
        dollar_amount = qty * prices[symbol]
        current_holdings[symbol] = dollar_amount / equity
    
    # Determine how much current_holdings needs to change to match desired holdings
    delta_holdings = {}
    all_symbols = set(list(current_holdings.keys()) + list(desired_holdings.keys()))

    for symbol in all_symbols:
        if symbol in current_holdings and symbol in desired_holdings:
            delta_holdings[symbol] = desired_holdings[symbol] - current_holdings[symbol]

        elif symbol in desired_holdings:
            delta_holdings[symbol] = desired_holdings[symbol]

        elif symbol in current_holdings:
            delta_holdings[symbol] = -current_holdings[symbol]
    
    # Convert the percentages of total account equity to shares and place trades
    for symbol, p_equity in delta_holdings.items():
        if symbol in prices:
            price = prices[symbol]
        n_shares = int(p_equity * equity / price)

        # If there is no need to trade anything we're done with this symbol
        if n_shares == 0:
            continue

        n_held = current_shares[symbol] if symbol in current_holdings else 0
            
        # If n_shares is negative we sell otherwise we buy
        side = "sell" if n_shares < 0 else "buy"
        print(symbol)

        # If we have a current holding and we are reducing our holding and that reduction is larger than our current holding
        if n_held != 0 and ((n_shares < 0) != (n_held < 0)) and abs(n_shares) > abs(n_held):
            # Close our current holding
            alp_trader.place_market_order(symbol, abs(n_held), side)
            # Open the remaining shares
            if n_shares - n_held != 0:
                alp_trader.place_market_order(symbol, abs(n_shares) - abs(n_held), side)
        else:
            # Otherwise just trade the shares
            alp_trader.place_market_order(symbol, abs(n_shares), side)
    

In [47]:
KEYS_PATH = "alpaca_config.json" # Path to Alpaca Keys in user system
alp_trader = AlpacaTrader(KEYS_PATH, is_paper=True)
alp_data = AlpacaData(KEYS_PATH)

In [61]:
update_holdings(alp_trader, {
    "SPY": 0.2,
    "SPXS": 0.3,
    "TSLA": 0.5
})

SPXS


In [14]:
desired_holdings = {
    "SPY": -0.2,
    "SPXS": 0.4
}

In [11]:
equity = float(alp_trader.get_account().json()["equity"])

In [45]:
positions = alp_trader.get_positions().json()
positions

[{'asset_id': 'b28f4066-5c6d-479b-a2af-85dc1a8f16fb',
  'symbol': 'SPY',
  'exchange': 'ARCA',
  'asset_class': 'us_equity',
  'asset_marginable': True,
  'qty': '-1',
  'avg_entry_price': '434.33',
  'side': 'short',
  'market_value': '-434.43',
  'cost_basis': '-434.33',
  'unrealized_pl': '-0.1',
  'unrealized_plpc': '-0.0002302396795064',
  'unrealized_intraday_pl': '-0.1',
  'unrealized_intraday_plpc': '-0.0002302396795064',
  'current_price': '434.43',
  'lastday_price': '434.04',
  'change_today': '0.0008985346972629'}]

In [42]:
prices = {}
for position in positions:
    prices[position["symbol"]] = float(position["current_price"])

for symbol in desired_holdings:
    if symbol not in prices:
        prices[symbol] = float(alp_trader.get_current_price(symbol).json()["latestTrade"]["p"])

print(prices)

{'SPY': 435.14, 'SPXL': 111.71, 'SPXS': 22.49}


In [44]:
current_holdings = {}
for position in positions:
    symbol = position["symbol"]
    dollar_amount = float(position["qty"]) * prices[symbol]
    current_holdings[symbol] = dollar_amount / equity
current_holdings

{'SPY': 0.044360839554935805, 'SPXL': 0.0011388402322659098}

In [18]:
delta_holdings = {}
all_symbols = set(list(current_holdings.keys()) + list(desired_holdings.keys()))

for symbol in all_symbols:
    if symbol in current_holdings and symbol in desired_holdings:
        delta_holdings[symbol] = desired_holdings[symbol] - current_holdings[symbol]

    elif symbol in desired_holdings:
        delta_holdings[symbol] = desired_holdings[symbol]

    elif symbol in current_holdings:
        delta_holdings[symbol] = -current_holdings[symbol]
delta_holdings

{'SPY', 'SPXL', 'SPXS'}


{'SPY': -0.24436083955493582, 'SPXL': -0.0011388402322659098, 'SPXS': 0.4}

In [40]:
for symbol, p in delta_holdings.items():
    if symbol in prices:
        price = prices[symbol]
    n_shares = int(p * equity / price)
    print(symbol, n_shares)

SPY -55
SPXL -1
SPXS 1746
